## Bigquery Exercise
If a NCAA basketball team is at home and behind 6 points with only 6 minutes remaining in the game, what is the probability that the team will comeback and win the game?

https://bigquery.cloud.google.com/table/bigquery-public-data:ncaa_basketball

In [233]:
# #sample sql code 
# query="""
# SELECT
#   weight_pounds,
#   is_male,
#   mother_age,
#   mother_race,
#   plurality,
#   gestation_weeks,
#   mother_married,
#   ever_born,
#   cigarette_use,
#   alcohol_use,
#   FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
# FROM
#   publicdata.samples.natality
# WHERE year > 2000
# """
# import google.datalab.bigquery as bq
# df = bq.Query(query + " LIMIT 100").execute().result().to_dataframe()
# df.head()

In [238]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np

In [239]:
player_team = """
SELECT full_name, team_id
FROM `bigquery-public-data.ncaa_basketball.mbb_players_games_sr`
LIMIT 100
"""
player_team = bq.Query(player_team).execute().result().to_dataframe()
player_team.head()

,full_name,team_id
0,Nick Mersch,0496c4c9-37a7-48db-ae81-c42eeb2de5fc
1,John Noce,0496c4c9-37a7-48db-ae81-c42eeb2de5fc
2,Kevin Hyland,0496c4c9-37a7-48db-ae81-c42eeb2de5fc
3,Christian Comenge,0496c4c9-37a7-48db-ae81-c42eeb2de5fc
4,Britton Goldsmith,0717335e-2875-4009-b0ff-3b7b0399ff8a


In [242]:
#%%bq query -n cyclesharing
#   ` -> wild card 
query = """
SELECT game_id, home_id, away_id, possession_team_id, points_scored
 FROM `bigquery-public-data.ncaa_basketball.mbb_pbp_sr` 
  WHERE  elapsed_time_sec  <= (40-6)*60  AND period = 2  AND possession_team_id IS NOT NULL AND points_scored IS NOT NULL
"""

df = bq.Query(query).execute().result().to_dataframe()
df.head()

,game_id,home_id,away_id,possession_team_id,points_scored
0,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,2.0
1,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,2.0
2,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,2.0
3,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,1.0
4,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,2.0


In [243]:
def home_or_away_score(row):
    if row['possession_team_id'] == row['home_id']:
      return 'away'
    else: 
      return 'home'
  
df['home_or_away_score'] = df.apply(home_or_away_score, axis=1)
df.head()

,game_id,home_id,away_id,possession_team_id,points_scored,home_or_away_score
0,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,2.0,home
1,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,2.0,home
2,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,2.0,home
3,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,1.0,home
4,07e3e7a6-e178-4610-a8e7-51ab0f5df03e,18e89867-9201-41ce-ba19-fadddf92fa17,0095032d-6143-44f2-8974-f6815fc56c5b,0095032d-6143-44f2-8974-f6815fc56c5b,2.0,home


In [244]:
grouped_df = df.groupby( [ "game_id", "home_or_away_score"]).sum()
grouped_df.head(10) 

points_scored
game_id                              home_or_away_score               
000872e5-f02a-4b64-ac73-b6b1a7ad10ec away                         17.0
                                     home                         15.0
000918c3-b8bf-472a-9a12-94a1984700eb away                         29.0
                                     home                         20.0
000b3698-3ce9-44bd-954b-65703a2c6ef1 away                         18.0
                                     home                         12.0
00114857-a305-499f-95d2-2b1762266681 away                         17.0
                                     home                         14.0
0012eab2-237c-4df1-846b-36430530867f away                         26.0
                                     home                         24.0

In [245]:
grouped_df.reset_index(inplace=True)

In [246]:
grouped_df.head(10)

,game_id,home_or_away_score,points_scored
0,000872e5-f02a-4b64-ac73-b6b1a7ad10ec,away,17.0
1,000872e5-f02a-4b64-ac73-b6b1a7ad10ec,home,15.0
2,000918c3-b8bf-472a-9a12-94a1984700eb,away,29.0
3,000918c3-b8bf-472a-9a12-94a1984700eb,home,20.0
4,000b3698-3ce9-44bd-954b-65703a2c6ef1,away,18.0
5,000b3698-3ce9-44bd-954b-65703a2c6ef1,home,12.0
6,00114857-a305-499f-95d2-2b1762266681,away,17.0
7,00114857-a305-499f-95d2-2b1762266681,home,14.0
8,0012eab2-237c-4df1-846b-36430530867f,away,26.0
9,0012eab2-237c-4df1-846b-36430530867f,home,24.0


In [247]:
game_id = grouped_df.iloc[::2, 0].reset_index(drop=True)
game_id.head()

0    000872e5-f02a-4b64-ac73-b6b1a7ad10ec
1    000918c3-b8bf-472a-9a12-94a1984700eb
2    000b3698-3ce9-44bd-954b-65703a2c6ef1
3    00114857-a305-499f-95d2-2b1762266681
4    0012eab2-237c-4df1-846b-36430530867f
Name: game_id, dtype: object

In [248]:
away_score = grouped_df.iloc[::2, 2].reset_index(drop=True)
away_score.head()

0    17.0
1    29.0
2    18.0
3    17.0
4    26.0
Name: points_scored, dtype: float64

In [249]:
home_score = grouped_df.iloc[1::2, 2].reset_index(drop=True)
home_score.head()

0    15.0
1    20.0
2    12.0
3    14.0
4    24.0
Name: points_scored, dtype: float64

In [250]:
diff = np.subtract(away_score, home_score).reset_index(drop=True)
diff.head()

0    2.0
1    9.0
2    6.0
3    3.0
4    2.0
Name: points_scored, dtype: float64

In [251]:
finale = pd.DataFrame({"game_id" : game_id, "diff" : diff})
finale = finale[finale['diff']==6].reset_index(drop=True)
finale.head()

,diff,game_id
0,6.0,000b3698-3ce9-44bd-954b-65703a2c6ef1
1,6.0,00323329-7205-46c6-9add-26d1422d4b73
2,6.0,00488111-6f5a-406c-ada5-4dc4b4c9a2f5
3,6.0,00c9a08b-dc98-45f9-811f-998807ffc53d
4,6.0,01571a1e-306f-42e8-8f19-f8ec4ffe9cde


In [252]:
total_matches = len(finale)
total_matches

350

In [253]:
home_win = """
  SELECT game_id
FROM `bigquery-public-data.ncaa_basketball.mbb_games_sr`
WHERE h_points_game > a_points_game
"""

home_win = bq.Query(home_win).execute().result().to_dataframe()
home_win.head()

,game_id
0,d6617923-0b23-49e4-af9b-9e4d0243e45c
1,ffb463a4-dd3c-4ed9-b503-311b95ef0295
2,d9cf6c1e-2125-4dbe-9c34-d0f8de911b10
3,c4e04822-4888-4ac1-b79f-183b9be8060f
4,dab342a9-2f3b-4d23-90ef-3ab447df03d4


In [254]:
merged_Frame = pd.merge(home_win,finale, on = 'game_id')
len(merged_Frame)

190

In [255]:
print("The probability is",round(190/350, 2)*100,"%")

The probability is 54.0 %
